In [9]:
import pandas as pd
from hacktops.model import SimpleDTWWindowEvaluator_Spark, TopFinder
from hacktops.utils import get_true_windows
from dtaidistance.dtw import distance as dtai_dtw
from tslearn.metrics import dtw as tslearn_dtw
import random
from plotly.offline import iplot

top_ = 'CONRAD'
df_logs_ = pd.read_parquet("../data/logs.parquet")
df_loc_ = pd.read_parquet("../data/loc.parquet")
df_tops_ = pd.read_parquet("../data/tops.parquet")

In [2]:
true_windows4train = get_true_windows(df_logs_, df_tops_, top_)

NAN FOUND
NAN FOUND
NAN FOUND
NAN FOUND
119196 windows extracted from 600 wells
X: (119196, 61)
y: (119196,)
8940 true windows left


In [3]:

import os
os.environ["PYSPARK_PYTHON"] = '/usr/users/bdmagr4/luo/bdrp/hacktops/env/bin/python3'
os.environ["PYSPARK_DRIVER_PYTHON"] = '/usr/users/bdmagr4/luo/bdrp/hacktops/env/bin/python3'

from pyspark import SparkContext
sc = SparkContext("local[16]", "First App")

22/11/30 15:55:02 WARN Utils: Your hostname, kyle40 resolves to a loopback address: 127.0.1.1; using 172.20.10.140 instead (on interface enp24s0f1)
22/11/30 15:55:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/30 15:55:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
simple_dtw_model = SimpleDTWWindowEvaluator_Spark(sc, metric=tslearn_dtw)
simple_dtw_model.fit(true_windows4train)

top_finder = TopFinder(simple_dtw_model, top_)
top_finder.examine_dataset(df_tops_)

In [5]:
test_well_name = 512333171.0
df_test_well = df_logs_[df_logs_['wellName'] == test_well_name]

In [6]:
top_finder.find_top(df_test_well)

1780 candidate windows


6692.5

In [10]:
from hacktops.utils import get_true_depth, visual_scores
true_depth = get_true_depth(test_well_name, top_, df_tops_)
depth = [w[0] for w in top_finder.windows]
fig = visual_scores(depth, top_finder.scores, top_finder.top_depth, true_depth, test_well_name)
iplot(fig)